<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/main/30_num_int/00_zeroth_order.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
# 그래프, 수학 기능 추가
# Add graph and math features
import pylab as py



# 수치 적분<br>Numerical Integration



여기서는 정적분의 근사값을 수열의 합으로 계산할 것이다.<br>
Here, we would approximate the definite integral by a sum of a sequence.



$$
\int_{x=a}^{x=b}f(x)dx \approx \sum_{k=0}^{n-1}F_k
$$


* 경우에 따라, $f(x)$의 부정적분을 구하는 것이 곤란할 수 있다.<br>Sometimes, it is not easy to find the indefinite integral of $f(x)$.
* $f(x)$ 함수는 알지 못하고 함수의 값만 주어지는 경우도 있다.<br>Also sometimes, we do not know $f(x)$; only some data points of $f(x)$ are available.



면적이 2인 원의 반지름을 구해 보자.<br>Let's find the radius of a circle with area of 2.



$$
\begin{align}
    \pi r^2 &= 2 \\
    r^2 &= \frac{2}{\pi} \\
    r &= \sqrt{\frac{2}{\pi}}
\end{align}
$$



In [ ]:
r = py.sqrt(2.0 / py.pi)



In [ ]:
r



이러한 원의 중심이 원점에 위치하고 있다고 생각해 보자.<br>Let's assume that a circle of such radius has its center at the origin.



$$
x^2 + y^2 = r^2 \\
y^2 = r^2 - x^2 \\
y = \pm \sqrt{r^2 - x^2} \\
y_{plus} = + \sqrt{r^2 - x^2} \\
y_{minus} = - \sqrt{r^2 - x^2}
$$



In [ ]:
x_array = py.linspace(-r, r, 64)
y_plus = py.sqrt(r**2 - x_array ** 2)
py.plot(x_array, y_plus, '.-')

x_array2 = r * py.cos(py.deg2rad(py.linspace(180, 0, 16)))
y_minus = -py.sqrt(r**2 - x_array2 ** 2)
py.plot(x_array2, y_minus, '.-')

py.axis('equal')
py.grid(True)



$+$ 부분만 생각하기로 하자.<br>Let's just think about the $+$ side only.



$$
y = \sqrt{r^2 - x^2}
$$



In [ ]:
x_array = py.sort(r * py.cos(py.deg2rad(range(0, 180))))
y_plus = py.sqrt(r**2 - x_array ** 2)

py.fill_between(x_array, y_plus)

py.axis('equal')
py.grid(True)



이 반원의 면적을 수치적으로 구해보기로 하자.  (반원의 정확한 값은 얼마이겠는가?)<br>
Let's try to numerically find the area of this half-circle. (What would be the exact value?)



## 0차 적분<br>0th Order Integration



우선 $x$를 일정 간격으로 나누어 보자.<br>Let's divide the $x$ coordinates in a constant interval.



In [ ]:
d = r * 2.0

n = 10

x_array_bar = py.linspace(-r, r, n+1)
y_array_bar = py.sqrt(abs(r**2 - x_array_bar ** 2))
delta_x = x_array_bar[1]-x_array_bar[0]

py.fill_between(x_array, y_plus)
# TODO : 막대그래프 직사각형 안을 칠하지 않으려면 어떻게 하면 좋겠는가?
# TODO : How can we remove the color inside the rectangle?
py.bar(x_array_bar, y_array_bar, width=delta_x, alpha=0.5, align='edge', edgecolor='k')

py.axis('equal')
py.grid(True)



아래 셀은 `x_array` 간격을 확인한다.<br>
Following cell verifies increment of `x_array`.



In [ ]:
assert 1e-3 > abs(delta_x - (d/n)), (delta_x, d/n)



직사각형의 모양과 반원의 모양이 정확히 일치하지는 않는다는 점을 기억하자.<br>
Let's remember that the areas of the rectangles and the half circle are not exactly the same.



각 직사각형의 면적을 구해서 더해 보자<br>Let's find the area of each rectangle and sum up.



$$
    Area = \sum_{k=0}^{n-1} F_k
$$



$$
    F_k = f(x_k)\cdot \Delta x
$$



$$
    Area = \sum_{k=0}^{n-1} f(x_k)\cdot \Delta x
$$



In [ ]:
summation = 0

for k in range(n):
    F_k = y_array_bar[k] * delta_x
    print('k = %2d, F_k = %g' % (k, F_k))
    summation += F_k

print('summation =', summation)



예상한 값 1에 더 비슷한 값을 얻기 위해 더 잘게 나누어 보자<br>To obtain the result closer to the expected value of 1, let's divide with a narrower interval.



In [ ]:
n = 100

x_array_bar = py.linspace(-r, r, n+1)

y_array_bar = py.sqrt(r**2 - x_array_bar ** 2)
delta_x = x_array_bar[1]-x_array_bar[0]

py.fill_between(x_array, y_plus)
py.bar(x_array_bar, y_array_bar, width=delta_x, alpha=0.5, align='edge', edgecolor='k')

py.axis('equal')
py.grid(True)



In [ ]:
summation = 0

for k in range(n):
    summation += delta_x * y_array_bar[k]

print('summation =', summation)



더 잘게 나눈 결과에 대한 의견은 어떠한가?<br>
What is your opinion about using the narrower partitions?



### 함수로 구현<br>Implement in a Function



다양한 경우에 더 편리하게 적용하기 위해 함수 형태로 만들어 보자.<br>To make it more convenient to apply to various cases, let's implement in a function



In [ ]:
def half_circle(x):
    return py.sqrt(r**2 - x ** 2)



In [ ]:
def get_delta_x(xi, xe, n):
    return (xe - xi) / n



In [ ]:
def num_int_0(f, xi, xe, n, b_verbose=False):
    x_array = py.linspace(xi, xe, n+1)

    delta_x = x_array[1] - x_array[0]

    assert 1e-3 > (abs(delta_x - get_delta_x(xi, xe, n)) / get_delta_x(xi, xe, n)), f"delta_x = {delta_x}"
    
    integration_result = 0.0
    
    for k in range(n):
        x_k = x_array[k]
        F_k = f(x_k) * delta_x

        if b_verbose: 
            print('k = %2d, F_k = %g' % (k, F_k))

        integration_result += F_k
    
    return integration_result



In [ ]:
n = 100
result = num_int_0(half_circle, -r, r, n)
print('result =', result)



In [ ]:
%timeit -n 100 result = num_int_0(half_circle, -r, r, n)



### $cos \theta$의 반 주기<br>Half period of $cos \theta$



$$
\begin{align}
I &= \int_{\theta=0}^{\theta=\pi}cos(\theta)d\theta \\
  &= \left[sin(\theta) \right]_{\theta=0}^{\theta=\pi} \\
  &= 0
\end{align}
$$



In [ ]:
theta_deg = py.arange(180+1)
theta_rad = py.deg2rad(theta_deg)
s = py.sin(theta_rad)
c = py.cos(theta_rad)

py.fill_between(theta_deg, c, color="C1", label="cos")

x_array_bar = py.linspace(0, 180, 10+1)
y_array_bar = py.cos(py.deg2rad(x_array_bar))
py.bar(x_array_bar[:-1], y_array_bar[:-1], width=x_array_bar[1] - x_array_bar[0], alpha=0.5, align='edge', edgecolor='k')

py.xticks(x_array_bar)

py.plot(theta_deg, s, color="C0", label="sin")

py.xlabel(r"$\theta(deg)$")

py.grid(True)
py.legend(loc=0);



In [ ]:
n = 10
result_cos = num_int_0(py.cos, 0, py.pi, n, b_verbose=True)
print('result =', result_cos)



In [ ]:
theta_deg = py.arange(180+1)
theta_rad = py.deg2rad(theta_deg)
s = py.sin(theta_rad)
c = py.cos(theta_rad)

py.fill_between(theta_deg, c, color="C1", label="cos")

x_array_bar = py.linspace(0, 180, 11+1)
y_array_bar = py.cos(py.deg2rad(x_array_bar))
py.bar(x_array_bar[:-1], y_array_bar[:-1], width=x_array_bar[1] - x_array_bar[0], alpha=0.5, align='edge', edgecolor='k')

py.xticks(x_array_bar)

py.xlabel(r"$\theta(deg)$")

py.grid(True)
py.legend(loc=0);



In [ ]:
n = 11
result_cos = num_int_0(py.cos, 0, py.pi, n, b_verbose=True)
print('result =', result_cos)



In [ ]:
theta_deg = py.arange(180+1)
theta_rad = py.deg2rad(theta_deg)
s = py.sin(theta_rad)
c = py.cos(theta_rad)

py.fill_between(theta_deg, c, color="C1", label="cos")

x_array_bar = py.linspace(0, 180, 100+1)
y_array_bar = py.cos(py.deg2rad(x_array_bar))
py.bar(x_array_bar[:-1], y_array_bar[:-1], width=x_array_bar[1] - x_array_bar[0], alpha=0.5, align='edge', edgecolor='k')

py.xticks(x_array_bar[::10])

py.xlabel(r"$\theta(deg)$")

py.grid(True)
py.legend(loc=0);



In [ ]:
n = 100
result_cos = num_int_0(py.cos, 0, py.pi, n)
print('result =', result_cos)



What do you think about the differences between the exact and numerical solutions?<br>엄밀해와 수치해의 차이에 대한 의견은?



### 1/4 원<br>A quarter circle



In [ ]:
n = 10
result_quarter = num_int_0(half_circle, -r, 0, n, b_verbose=True)
print('result =', result_quarter)



In [ ]:
n = 10
result_quarter = num_int_0(half_circle, 0, r, n, b_verbose=True)
print('result =', result_quarter)



In [ ]:
n = 100
result_quarter = num_int_0(half_circle, -r, 0, n)
print('result =', result_quarter)



## 연습 문제<br>Exercises



도전 과제 1: $e^ x$ 를 $0 \le x \le 1$ 구간에서 0차 적분으로 적분하시오.  이론적 엄밀해와 비교하시오.<br>Try this 1: Integrate  $e^ x$ over $0 \le x \le 1$ interval using the 0th order integration.  Compare with the exact solution.<br>



도전 과제 2: $sin \left( cos x \right)$ 를 $0 \le x \le 1$ 구간에서 적분하시오.<br>
Try this 2: Integrate $sin \left( cos x \right)$ over $0 \le x \le 1$ interval. <br>
(ref : [Examples for
Numerical Integration](https://www.wolframalpha.com/examples/mathematics/applied-mathematics/numerical-analysis/numerical-integration/), Wolfram Alpha, Accessed Aug 28 2018)



도전 과제 3: 이미 다루었던 이분법 함수와 0차 적분을 이용하여 면적이 2인 원의 반지름을 구하는 프로그램을 작성하고 실행해 보시오.<br>
Try this 3: Using the bisection method function and 0th roder intergration, write a program finding radius of a circle with area of two, and run it.



## 리만 합<br>Riemann Sum



이렇게 어떤 함수의 정적분을 유한한 합으로 바꾸어 계산하는 것을 리만 합이라고 부른다.<br>
Riemanm Sum is a type of approximation of an integral by a finite sum. [[wikipedia](https://en.wikipedia.org/wiki/Riemann_sum)]



## `pylab.linspace()`



일정 간격의 배열을 생성한다.<br>
This would generate an array of a constant interval.



In [ ]:
py.linspace(0, 10, 5)



아래 셀의 결과를 비교해 보시오.<br>
Compare the results of the following cells.



In [ ]:
py.arange(0, 10+1, 1)



In [ ]:
py.linspace(0, 10, 10+1)



## 함수형 프로그래밍<br>Functional programming



간격이 일정하다면 면적의 근사값을 다음과 같이 바꾸어 쓸 수 있다.<br>
If the interval $\Delta x$ is constant, we may rewrite the approximation of the area as follows.



$$
    Area = \sum_{k=0}^{n-1} f(x_k)\cdot \Delta x= \Delta x \sum_{k=0}^{n-1} f(x_k) 
$$



할당문 없이 `sum()` 과 `map()` 함수로 구현해 보자.<br>
Instead of assignments, let's implement using `sum()` and `map()` functions.



In [ ]:
def num_int_0_functional(f, xi, xe, n):
    return (
        get_delta_x(xi, xe, n) * sum(
            map(
                f,
                py.linspace(xi, xe, n+1)[:-1],
            )
        )
    )



In [ ]:
n = 100
result_func = num_int_0_functional(half_circle, -r, r, n)
print('result_func =', result_func)



In [ ]:
assert 1e-3 > abs(result - result_func), f"result = {result}, result_func = {result_func}"



In [ ]:
%timeit -n 100 result_func = num_int_0_functional(half_circle, -r, r, n)



이와 같이 할당문과 부가적 효과 없이 함수로만 구현하는 형태를 *함수형 프로그래밍* 이라고 한다.<br>
*Functional programming* is to implement with functions only without assignments and side effects. [[Sahota](https://dev.to/navi/why-functional-programming-matters-2o95)]



## NumPy 벡터화<br>Vectorization of NumPy



In [ ]:
import pylab as py



In [ ]:
def num_int_0_vector(f, xi, xe, n):
    return f(
        py.linspace(xi, xe-get_delta_x(xi, xe, n), n)
    ).sum() * get_delta_x(xi, xe, n)



In [ ]:
n = 100
result_vect = num_int_0_vector(half_circle, -r, r, n)
print('result_vect =', result_vect)



In [ ]:
assert 1e-3 > abs(result - result_vect), f"result = {result}, result_vect = {result_vect}"



In [ ]:
%timeit -n 100 result_vect = num_int_0_vector(half_circle, -r, r, n)



## 시험<br>Test



아래는 함수가 맞게 작동하는지 확인함<br>
Following cells verify whether the functions work correctly.



In [ ]:
import pylab as py
r = py.sqrt(1.0 / py.pi)
n = 10


def half_circle(x):
    return py.sqrt(r**2 - x ** 2)


assert 0.25 > num_int_0(half_circle, -r, 0, n), num_int_0(half_circle, -r, 0, n)
assert 0.25 < num_int_0(half_circle, 0, r, n), num_int_0(half_circle, 0, r, n)
assert 0.25 > num_int_0_functional(half_circle, -r, 0, n), num_int_0_functional(half_circle, -r, 0, n)
assert 0.25 < num_int_0_functional(half_circle, 0, r, n), num_int_0_functional(half_circle, 0, r, n)
assert 0.25 > num_int_0_vector(half_circle, -r, 0, n), num_int_0_vector(half_circle, -r, 0, n)
assert 0.25 < num_int_0_vector(half_circle, 0, r, n), num_int_0_vector(half_circle, 0, r, n)



In [ ]:
import math

epsilon = 0.1

assert math.isclose(4.0 * num_int_0(half_circle, -r, 0, n)           , 1.0, abs_tol=epsilon)
assert math.isclose(4.0 * num_int_0(half_circle, 0, r, n)            , 1.0, abs_tol=epsilon)
assert math.isclose(4.0 * num_int_0_functional(half_circle, -r, 0, n), 1.0, abs_tol=epsilon)
assert math.isclose(4.0 * num_int_0_functional(half_circle, 0, r, n) , 1.0, abs_tol=epsilon)
assert math.isclose(4.0 * num_int_0_vector(half_circle, -r, 0, n)    , 1.0, abs_tol=epsilon)
assert math.isclose(4.0 * num_int_0_vector(half_circle, 0, r, n)     , 1.0, abs_tol=epsilon)



## Final Bell<br>마지막 종



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

